## Mortality Training
1. Load the MIMIC III Dataset
2. Normalize Data
3. Load into Pytorch Dataloader
4. Train
5. Evaluate

In [3]:
from comet_ml import Experiment
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from core.model import SAnD
from mimic3_benchmarks.mimic3benchmark.readers import InHospitalMortalityReader as Reader
from mimic3_benchmarks.mimic3models.preprocessing import Discretizer, Normalizer
from utils.ihm_utils import load_data
from utils.functions import get_weighted_sampler, get_weights
from utils.trainer import NeuralNetworkClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
task = "in-hospital-mortality"

## Load Data
Using the [MimicIII Benchmark's](https://github.com/YerevaNN/mimic3-benchmarks) InHospitalMortalityReader and associated functions, load the data into memory.

If you are following along you will need to arrange the data following the directions on the link.


In [4]:
train_reader = Reader(dataset_dir=f"mimic3_benchmarks/data/{task}/train",
                              listfile=f"mimic3_benchmarks/data/{task}/train_listfile.csv")
val_reader = Reader(dataset_dir=f"mimic3_benchmarks/data/{task}/train",
                              listfile=f"mimic3_benchmarks/data/{task}/val_listfile.csv")
test_reader = Reader(dataset_dir=f"mimic3_benchmarks/data/{task}/test",
                              listfile=f"mimic3_benchmarks/data/{task}/test_listfile.csv")

### Example data

In [13]:
ex = pd.DataFrame.from_dict(train_reader.read_example(0)['X'])
ex.columns = train_reader.read_example(0)["header"]
ex

,Hours,Capillary refill rate,Diastolic blood pressure,Fraction inspired oxygen,Glascow coma scale eye opening,Glascow coma scale motor response,Glascow coma scale total,Glascow coma scale verbal response,Glucose,Heart Rate,Height,Mean blood pressure,Oxygen saturation,Respiratory rate,Systolic blood pressure,Temperature,Weight,pH
0,0.2138888888888889,,71.0,,,,,,,94,,95,92.0,16,141.0,,,
1,0.46388888888888885,,71.0,,,,,,,91,,95,93.0,16,141.0,,,
2,0.7138888888888889,,,,,,,,,89,,,93.0,19,,,,
3,0.9638888888888889,,,,,,,,,89,,,94.0,13,,,,
4,1.1305555555555555,,,,,,,,,,,,,,,,,7.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,44.21388888888889,,58.0,,,,,,,103,,81,99.0,21,113.0,,,
67,45.21388888888889,,59.0,,4 Spontaneously,6 Obeys Commands,15,5 Oriented,,107,,81,97.0,22,116.0,,,
68,46.21388888888889,,61.0,,,,,,,100,,85,97.0,20,120.0,,,
69,47.21388888888889,,58.0,,,,,,,90,,83,100.0,18,123.0,37.0,,


### Normalize and Discretize data
From the MIMIC-III Benchmark repo

In [5]:
discretizer = Discretizer(timestep=1.0,
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')

discretizer_header = discretizer.transform(train_reader.read_example(0)["X"])[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

In [6]:
normalizer = Normalizer(fields=cont_channels)  # choose here which columns to standardize
normalizer_state = 'train/ihm_ts1.0.input_str-previous.start_time-zero.normalizer'
normalizer.load_params(normalizer_state)

In [ ]:
train_raw = load_data(train_reader, discretizer, normalizer)
val_raw = load_data(val_reader, discretizer, normalizer)
test_raw = load_data(val_reader, discretizer, normalizer)

N, seq_len, feature_count = train_raw[0].shape

In [7]:
batch_size = 256

train_ds = TensorDataset(train_raw[0], train_raw[1])
val_ds = TensorDataset(val_raw[0], val_raw[1])
test_ds = TensorDataset(test_raw[0], test_raw[1])

train_loader = DataLoader(train_ds, batch_size=batch_size)#sampler=get_weighted_sampler(y_train))
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size)

## Create model, Train, and Evaluate
Train the model, this will export results to Comet ML.
Creating is done in the same step such that a new experiment made each time
Evaluation is done in the same step such that the Comet ML experiment is also deleted.

In [8]:
n_heads = 8
factor = 12 # M
num_class = 2
num_layers = 2 # N
epochs = 30
betas = (0.9, 0.98)
lr = 0.0005
eps = 4e-09
weight_decay = 5e-4

experiment = Experiment(
    api_key="eQ3INeSsFGUYKahSdEtjhry42",
    project_name="general",
    workspace="samdoud"
)

clf = NeuralNetworkClassifier(
    SAnD(feature_count, seq_len, n_heads, factor, num_class, num_layers, dropout_rate=0.3),
    nn.CrossEntropyLoss(weight=torch.tensor(get_weights(train_raw[1], level=1), dtype=torch.float32).to(device=device)),
    optim.Adam, optimizer_config={
        "lr": lr, "betas": betas, "eps": eps, "weight_decay": weight_decay},
    experiment=experiment
)

clf.fit(
    {
        "train": train_loader,
        "val": val_loader
    },
    validation=True,
    epochs=epochs,
    verbose=True,
)

clf.evaluate(test_loader)
experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/samdoud/general/975b737db72048e9a0730bae7fced099

Training - Epochs: 001/030:  99%|█████████▉| 14592/14681 [00:04<00:00, 5969.89it/s]

Train finished. Accuracy: 0.6446 MSE: 0.3554 AUROC: 0.6383 AUPRC: 0.4490


Training - Epochs: 001/030: 100%|██████████| 14681/14681 [00:04<00:00, 3252.31it/s]


Validation finished. Accuracy: 0.4789 MSE: 0.1144 AUROC: 0.6697 AUPRC: 0.5690


Training - Epochs: 002/030:  99%|█████████▉| 14592/14681 [00:02<00:00, 5918.43it/s]

...

Training - Epochs: 030/030: 100%|██████████| 14681/14681 [00:02<00:00, 5750.56it/s]


Validation finished. Accuracy: 0.7831 MSE: 0.0476 AUROC: 0.6898 AUPRC: 0.4733


Evaluating: 100%|██████████| 3222/3222 [00:00<00:00, 11385.22it/s]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/samdoud/general/975b737db72048e9a0730bae7fced099
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_AUPRC [13]         : (0.4699000046633014, 0.523934503496798)
COMET INFO:     test_AUROC [13]         : (0.68517571313601, 0.705429292929293)
COMET INFO:     test_MSE [13]           : (0.216796875, 0.234375)
COMET INFO:     test_accuracy [13]      : (0.765625, 0.783203125)
COMET INFO:     test_loss [13]          : (1.9413737058639526, 26.193230867385864)
COMET INFO:     train_AUPRC [1740]      : (0.2807586733367984, 0.8437258126934984)
COMET INFO:     train_AUROC [1740]      : (0.46840676292731087, 0.9425013248542661)
COMET INFO:     train_MSE [1740]        : (0.046875, 0

Evaluation finished. Accuracy: 0.7831 MSE: 0.2169 AUROC: 0.6898 AUPRC: 0.4733


COMET INFO: Uploading metrics, params, and assets to Comet before program termination (may take several seconds)
COMET INFO: The Python SDK has 3600 seconds to finish before aborting...
